In [1]:
#Module installation
import numpy as np
import lightgbm as lgb
import pandas as pd
# from kaggle.competitions import twosigmanews #Comment out here because this dataset is exclusive in Kaggle kernel
import matplotlib.pyplot as plt
import random
from datetime import datetime, date
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time

In [2]:
#Comment out this chunk because the whole dataset is exclusive in Kaggle kernel

#Collecting Dataset
# from kaggle.competitions import twosigmanews
# env = twosigmanews.make_env()
# print('Finished collecting data!')

In [3]:
#Sorting and storing Stock data and news data respectively
# (market_train_df, news_train_df) = env.get_training_data() #This is for Kaggle kernel version only
market_train_df = pd.read_csv("marketdata_sample.csv")
news_train_df = pd.read_csv("news_sample.csv")


In [4]:
news_train_df.head()

,time,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,...,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,2007-01-01 04:29:32+00:00,e58c6279551b85cf,China's Daqing pumps 43.41 mln tonnes of oil i...,3,1,RTRS,"{'CRU', 'CN', 'RTRS', 'ENR', 'LEN', 'EMRG', 'N...","{'O', 'Z', 'OIL'}",...,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,2007-01-01 07:03:34+00:00,2007-01-01 07:03:34+00:00,5a31c4327427f63f,"FEATURE-In kidnapping, finesse works best",3,1,RTRS,"{'BD', 'INS', 'LATAM', 'CA', 'US', 'MX', 'IL',...","{'PGE', 'PCU', 'PCO', 'DNP', 'MD', 'E', 'G', '...",...,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,2007-01-01 11:29:56+00:00,1cefd27a40fabdfe,PRESS DIGEST - Wall Street Journal - Jan 1,3,1,RTRS,"{'IQ', 'RO', 'US', 'ID', 'RET', 'RTRS', 'ENR',...","{'M', 'PMF', 'DNP', 'PTD', 'T', 'E', 'D', 'PSC...",...,0,0,0,0,0,0,0,5,11,17
3,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'PUB', 'BUS', 'INS', 'CA', 'ENT', 'US', 'FIN'...","{'M', 'PMF', 'DNP', 'PTD', 'T', 'E', 'D', 'PSC...",...,0,0,0,0,0,0,0,5,13,15
4,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3,1,RTRS,"{'PUB', 'BUS', 'INS', 'CA', 'ENT', 'US', 'FIN'...","{'M', 'PMF', 'DNP', 'PTD', 'T', 'E', 'D', 'PSC...",...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Comment out this chunk due to the difference reading dataset from Kaggle environment and reading directly from .csv file

#Reformat "time" to date
# market_train_df['time'] = market_train_df['time'].dt.date
#Save data only for 2010
# market_train_df = market_train_df.loc[market_train_df['time']>=date(2010, 1, 1)]

In [6]:
market_train_df.head()

,time,assetCode,assetName,universe,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,1.0,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,0.0,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1.0,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,1.0,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1.0,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994


In [7]:
#Importing modules for parallel work
from multiprocessing import Pool

#create_lag (average maximum minimum standard deviation creation method for moving average)
#Default values are included in method definition brackets

def create_lag(df_code,n_lag=[3,7,14,],shift_size=1):

    #Put df_code's asset code in code without duplication
    code = df_code['assetCode'].unique()
    
    #From return_features (revenue function) to col variable
    for col in return_features:
        #Loop n_lag to window variable
        for window in n_lag:
            #The value of the window variable is shifted from the present by the previous moving average value
            rolled = df_code[col].shift(shift_size).rolling(window=window)
            #Averages of moving average
            lag_mean = rolled.mean()
            #Maximal
            lag_max = rolled.max()
            #Minimal
            lag_min = rolled.min()
            #Standard Deviation
            lag_std = rolled.std()
            #Storing in df_code by the column name of col's value of _lag_window's value _mean
            df_code['%s_lag_%s_mean'%(col,window)] = lag_mean
            df_code['%s_lag_%s_max'%(col,window)] = lag_max
            df_code['%s_lag_%s_min'%(col,window)] = lag_min
            #Leaving STD out for now
#             df_code['%s_lag_%s_std'%(col,window)] = lag_std
   
    #If there is a null value in df_code, return it with -1
    return df_code.fillna(-1)

In [8]:
#generate_lag_features
def generate_lag_features(df,n_lag = [3,7,14]):
    #Put all the stock price data in features (feature variable)
    features = ['time', 'assetCode', 'assetName', 'volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10', 'universe']
    #Duplicate deletion of assetCode (stock code)
    assetCodes = df['assetCode'].unique()
    print(assetCodes)
    #Initiating all_df
    all_df = []
    #Stored in df_codes as aggregated by assetCode (SQL group by)
    df_codes = df.groupby('assetCode')
    df_codes = [df_code[1][['time','assetCode']+return_features] for df_code in df_codes]
    #showing the total length of df_codes
    print('total %s df'%len(df_codes))
    
    #Using 4 CPUs
    pool = Pool(4)
    #The result of substituting the argument of df_codes into the create_lag method and storing it in all_df
    all_df = pool.map(create_lag, df_codes)
    
    #Tie all_df to new_df
    new_df = pd.concat(all_df)
    #delete line with return_features empty value
    new_df.drop(return_features,axis=1,inplace=True)
    #Parallel processing end
    pool.close()
    return new_df

In [ ]:
#Setting return_features (revenue function) (eventually not just the closing price)
return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close']
#Setting the number of days for moving average
n_lag = [3,7,14]
#Time series feature extraction method to stock price training data, 3, 7, 14 moving average averaged
new_df = generate_lag_features(market_train_df,n_lag=n_lag)

['A.N' 'AAI.N' 'AAP.N' 'AAPL.O' 'ABB.N' 'ABC.N' 'ABD.N' 'ABM.N' 'ABT.N'
 'ABV.N' 'ABY.N' 'ACF.N' 'ACH.N' 'ACL.N' 'ACS.N' 'ACTS.O' 'ACXM.O' 'AD.N'
 'ADBE.O' 'ADCT.O' 'ADI.N' 'ADM.N' 'ADP.N' 'ADS.N' 'ADSK.O' 'ADTN.O'
 'AEA.N' 'AEE.N' 'AEG.N' 'AEIS.O' 'AEL.N' 'AEOS.O' 'AEP.N' 'AER.N' 'AES.N'
 'AET.N' 'AF.N' 'AFFX.O' 'AFG.N' 'AFL.N' 'AFR.N' 'AG.N' 'AGE.N' 'AGN.N'
 'AGU.N' 'AH.N' 'AHG.N' 'AHL.N' 'AHS.N' 'AHT.N' 'AIN.N' 'AINV.O' 'AIR.N'
 'AIT.N' 'AIV.N' 'AIZ.N' 'AJG.N' 'AKAM.O' 'AKR.N' 'AKS.N' 'ALB.N' 'ALD.N'
 'ALGN.O' 'ALK.N' 'ALL.N' 'ALNY.O' 'ALO.N' 'ALTR.O' 'ALU.N' 'ALV.N'
 'ALXN.O' 'AMAT.O' 'AMD.N' 'AME.N' 'AMED.O' 'AMG.N' 'AMGN.O' 'AMKR.O'
 'AMLN.O' 'AMMD.O' 'AMP.N' 'AMT.N' 'AMTD.O' 'AMX.N' 'AMZN.O' 'AN.N'
 'ANDE.O' 'ANDW.O' 'ANF.N' 'ANR.N' 'AOC.N' 'AOS.N' 'APA.N' 'APC.N' 'APD.N'
 'APH.N' 'APKT.O' 'APOL.O' 'APPB.O' 'AQNT.O']
total 100 df


In [ ]:
new_df.head()

In [ ]:
#The moving average data is added to the stock price training data by time and the outer join by the assetCode column
market_train_df = pd.merge(market_train_df,new_df,how='left',on=['time','assetCode'])

In [ ]:
#Check if moving average is properly added
print(market_train_df.columns)

In [ ]:
market_train_df.head()

In [ ]:
#mis_impute method (null value substitution)
def mis_impute(data):
    #Pull columns one by one
    for i in data.columns:
        #If the type is a string, replace the null value with other
        if data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        #If the type is numeric, empty value is replaced with average value
        elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        else:
            pass
    return data
#Replace stock price training data with reshaped data
market_train_df = mis_impute(market_train_df)

In [ ]:
#reshape data_prep
def data_prep(market_train):
    #While deletion of stock code duplication, pull one by one column while adding line number
    lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
    #Processing the stock code as above (probably adding line number to assetCodeT and putting it there)
    market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    #Delete null value row
    market_train = market_train.dropna(axis=0)
    return market_train
#Execute the above method
market_train_df = data_prep(market_train_df)
#shape indicates how many rows of data there are vertically and horizontally
print(market_train_df.shape)

In [ ]:
#You can see that assetCodeT is falling apart
market_train_df.tail()

In [ ]:
#Import modules to change data to numbers
from sklearn.preprocessing import LabelEncoder
#Only those that stock price stagnated or increased after ten days are thrown into the up variable
up = market_train_df['returnsOpenNextMktres10'] >= 0

#The value of universe (this can not be used as training data except 1) is thrown into the universe variable
universe = market_train_df['universe'].values


In [ ]:
universe

In [ ]:
#Dump into time variable
d = market_train_df['time']
#Fianl selection of training data
#Pull out columns one by one
#The following columns are designed to be excluded
fcol = [c for c in market_train_df if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'time_x','provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'universe','sourceTimestamp']]
fcol

In [ ]:
#Pull down the value of the column finally selected above
X = market_train_df[fcol].values
X

In [ ]:
#Set the value of TrueFalse of returnsOpenNextMktres10
up = up.values
up

In [ ]:
#r is not TrueFalse but throws the actual value
r = market_train_df.returnsOpenNextMktres10.values
r

In [ ]:
# Setting X value's range
#Minimum value for each column
mins = np.min(X, axis=0)
mins

In [ ]:
#Maximum value of each column
maxs = np.max(X, axis=0)
maxs

In [ ]:
#Calculating the range between maximal and minimal
rng = maxs - mins
rng

In [ ]:
#Insert X in how far it is from the lowest value
X = 1 - ((maxs - X) / rng)
X

In [ ]:
#Validate consistency
assert X.shape[0] == up.shape[0] == r.shape[0]
print(X.shape[0])
print(up.shape[0])
print(r.shape[0])
#Confirm that the number of each list matches

In [ ]:
#Inserting Xgboost here
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time

In [ ]:
#Work to separate training data and test data
X_train, X_test, up_train, up_test, r_train, r_test,u_train,u_test,d_train,d_test = \
model_selection.train_test_split(X, up, r,universe,d, test_size=0.25, random_state=99)

In [ ]:
#Setting up the LightGBM training data
train_data = lgb.Dataset(X, label=up.astype(int))
#Setting test data of LightGBM
test_data = lgb.Dataset(X_test, label=up_test.astype(int))

In [ ]:
# Setting LightGBM parameters 
x_1 = [0.19000424246380565, 2452, 212, 328, 202]
x_2 = [0.19016805202090095, 2583, 213, 312, 220]
print(up_train)

def exp_loss(p,y):
    y = y.get_label()
    grad = -y*(1.0-1.0/(1.0+np.exp(-y*p)))
    hess = -(np.exp(y*p)*(y*p-1)-1)/((np.exp(y*p)+1)**2)
    
    return grad,hess
###########################################
#Parameter tuning of actual machine learning
params_1 = {
        #settings for increasing accuracy
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
#         'objective': 'regression',
        #Learning degree (smaller value is better)
        'learning_rate': x_1[0],
        #Number of leaves (The larger the value, the better, but it is sometimes clogged if it is too big)
        'num_leaves': x_1[1],
        #Minimum number of leaves
        'min_data_in_leaf': x_1[2],
#         'num_iteration': x_1[3],
        #Number of iterations (larger value is better)
        'num_iteration': 239,
        #Increasing this value will increase accuracy but will be heavier. 
        'max_bin': x_1[4],
        #Setting of redundancy warning (0 is error output, 1 is log output, 2 is debugging)
        'verbose': 1
    }
#The second item is set to another value by the same method
params_2 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
#         'objective': 'regression',
        'learning_rate': x_2[0],
        'num_leaves': x_2[1],
        'min_data_in_leaf': x_2[2],
#         'num_iteration': x_2[3],
        'num_iteration': 172,
        'max_bin': x_2[4],
        'verbose': 1
    }

gbm_1 = lgb.train(params_1,
        train_data,
#It's not a big deal, but num_boost_round should be at least 200 and choose the fewest number of error rates for the test data.
#As for num_boost_round, there are some parts that are not convincing, so we will consider it later.
#After several tries, 133 times seems to be good.
        #Probably setting number of round of learning
        num_boost_round=133,
        #The location of the data for answers?
        valid_sets=test_data,
        #Minimal learning round
        early_stopping_rounds=5,
#         fobj=exp_loss,
        )
#Same thing below
gbm_2 = lgb.train(params_2,
        train_data,
        num_boost_round=133,
        valid_sets=test_data,
        early_stopping_rounds=5,
#         fobj=exp_loss,
        )

In [ ]:
#We average out the 1st data and the 2nd data
confidence_test = (gbm_1.predict(X_test) + gbm_2.predict(X_test))/2
confidence_test

In [ ]:
#Calculate the percentage of the predicted value
confidence_test = (confidence_test-confidence_test.min())/(confidence_test.max()-confidence_test.min())
confidence_test

In [ ]:
confidence_test = confidence_test*2-1
print(max(confidence_test),min(confidence_test))
confidence_test

In [ ]:
# Calculating the actual metrics used to calculate the final score
r_test = r_test.clip(-1,1) 
x_t_i = confidence_test * r_test * u_test
#Create a data frame with only dates and score values
data = {'day' : d_test, 'x_t_i' : x_t_i}
df = pd.DataFrame(data)
#We reorganize multidimensional arrays into one dimension by group by group (aggregate) by date
x_t = df.groupby('day').sum().values.flatten()
#Mean of score
mean = np.mean(x_t)
#Standard deviation of score
std = np.std(x_t)
#Reciprocal of coefficient of variation
score_test = mean / std
print(score_test)

In [ ]:
import gc
del X_train,X_test
gc.collect()

In [ ]:
#prediction
days = env.get_prediction_days()
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
total_market_obs_df = []
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    if (n_days%50==0):
        print(n_days,end=' ')
    t = time.time()
    market_obs_df['time'] = market_obs_df['time'].dt.date
    
    return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close']
    total_market_obs_df.append(market_obs_df)
    if len(total_market_obs_df)==1:
        history_df = total_market_obs_df[0]
    else:
        history_df = pd.concat(total_market_obs_df[-(np.max(n_lag)+1):])
    print(history_df)
    
    new_df = generate_lag_features(history_df,n_lag=[3,7,14])
    market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    
#     return_features = ['open']
#     new_df = generate_lag_features(market_obs_df,n_lag=[3,7,14])
#     market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    
    market_obs_df = mis_impute(market_obs_df)
    
    market_obs_df = data_prep(market_obs_df)
    
#     market_obs_df = market_obs_df[market_obs_df.assetCode.isin(predictions_template_df.assetCode)]
    
    X_live = market_obs_df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    
    t = time.time()
    lp = (gbm_1.predict(X_live) + gbm_2.predict(X_live))/2
    prediction_time += time.time() -t
    
    t = time.time()
    
    confidence = lp
    confidence = (confidence-confidence.min())/(confidence.max()-confidence.min())
    confidence = confidence * 2 - 1
    
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t
    
env.write_submission_file()
sub  = pd.read_csv("submission.csv")